### Analysis on Lunar Date Conversions
* At this moment, there are 2 algorithms available
  * Algo 1 is based on Hong Kong Observatory's data
  * Algo 2 is based on astro-calculation, using VSOP87D and ELP2000-82B
* The 2 algorithms procude different results on year 1914, 1915, 1916, 1920

In [1]:
from common import LunarAlgo, get_lunar_year_info, get_supported_lunar_year_range

In [2]:
get_lunar_year_info(LunarAlgo.ALGO_1, 1914)

LunarYearInfo(first_day=datetime.date(1914, 1, 26), leap_month=5, month_lengths=[30, 30, 29, 30, 29, 30, 29, 30, 29, 29, 30, 29, 30])

In [3]:
get_lunar_year_info(LunarAlgo.ALGO_2, 1914)

LunarYearInfo(first_day=datetime.date(1914, 1, 26), leap_month=5, month_lengths=[30, 30, 29, 30, 29, 30, 29, 30, 29, 30, 29, 29, 30])

In [4]:
from functools import partial

# Use algo1's supported range since it's narrower.
# The year supported by algo1 should also be supported by algo2.
supported_range = get_supported_lunar_year_range(LunarAlgo.ALGO_1)
years = list(range(supported_range.start, supported_range.end + 1))

f1 = partial(get_lunar_year_info, LunarAlgo.ALGO_1)
f2 = partial(get_lunar_year_info, LunarAlgo.ALGO_2)

results_algo1 = list(map(f1, years))
results_algo2 = list(map(f2, years))

In [5]:
from itertools import compress

results_cmp = [r1 == r2 for r1, r2 in zip(results_algo1, results_algo2)]

unmatched = list(compress(
  years, [not x for x in results_cmp]
))
unmatched

[1914, 1915, 1916, 1920]

In [6]:
for y in unmatched:
  algo1_info = get_lunar_year_info(LunarAlgo.ALGO_1, y)
  algo2_info = get_lunar_year_info(LunarAlgo.ALGO_2, y)

  print(f'Examine for year {y}')
  if algo1_info.first_day != algo2_info.first_day:
    print(f'  algo1: {algo1_info.first_day}')
    print(f'  algo2: {algo2_info.first_day}')
    print()

  if algo1_info.leap_month != algo2_info.leap_month:
    print(f'  algo1: {algo1_info.leap_month}')
    print(f'  algo2: {algo2_info.leap_month}')
    print()

  if algo1_info.month_lengths != algo2_info.month_lengths:
    print(f'  algo1: {algo1_info.month_lengths}')
    print(f'  algo2: {algo2_info.month_lengths}')
    print()

Examine for year 1914
  algo1: [30, 30, 29, 30, 29, 30, 29, 30, 29, 29, 30, 29, 30]
  algo2: [30, 30, 29, 30, 29, 30, 29, 30, 29, 30, 29, 29, 30]

Examine for year 1915
  algo1: [30, 29, 30, 30, 29, 30, 29, 30, 29, 30, 29, 29]
  algo2: [30, 29, 30, 30, 29, 30, 29, 30, 29, 30, 29, 30]

Examine for year 1916
  algo1: 1916-02-03
  algo2: 1916-02-04

  algo1: [30, 30, 29, 30, 29, 30, 30, 29, 30, 29, 30, 29]
  algo2: [29, 30, 29, 30, 29, 30, 30, 29, 30, 29, 30, 29]

Examine for year 1920
  algo1: [29, 30, 29, 29, 30, 29, 29, 30, 29, 30, 30, 30]
  algo2: [29, 30, 29, 29, 30, 29, 29, 30, 30, 29, 30, 30]



* According to https://ytliu0.github.io/ChineseCalendar/computation_simp.html, these calculation results are not accurate
  * 10th lunar month in 1914
  * 1st lunar month in 1916
  * 10th lunar month in 1920
* That's why algo1 and algo2 have different results on these years

In [7]:
import common
from datetime import timedelta

new_moon_moments = common.new_moons_in_year(1914).new_moon_moments

# Convert to UT1+8
new_moon_moments_ut1_8 = [m + timedelta(hours=8) for m in new_moon_moments]

# algo1 (observed): 1914.11.17
# algo2 (calculated): 1914.11.18
new_moon_moments_ut1_8[-2]

datetime.datetime(1914, 11, 18, 0, 1, 49, 741625)

In [8]:
new_moon_moments = common.new_moons_in_year(1916).new_moon_moments

# Convert to UT1+8
new_moon_moments_ut1_8 = [m + timedelta(hours=8) for m in new_moon_moments]

# algo1 (observed): 1916.02.03
# algo2 (calculated): 1916.02.04
new_moon_moments_ut1_8[1]

datetime.datetime(1916, 2, 4, 0, 5, 13, 706552)

In [9]:
new_moon_moments = common.new_moons_in_year(1920).new_moon_moments

# Convert to UT1+8
new_moon_moments_ut1_8 = [m + timedelta(hours=8) for m in new_moon_moments]

# algo1 (observed): 1920.11.10
# algo2 (calculated): 1920.11.11
new_moon_moments_ut1_8[-2]

datetime.datetime(1920, 11, 11, 0, 4, 48, 680642)

In [10]:
from common import Jieqi, jieqi_moment
from dataclasses import dataclass

# As per https://ytliu0.github.io/ChineseCalendar/computation_simp.html,
# The following Jieqi moments are near 00:00:00, and are different than the ones that the government provides.
# The differences of Jieqi moments also makes differences in Ganzhi calendar, since Ganzhi calendar is based on Jieqi.

@dataclass
class JieqiPair:
  year: int
  jq: Jieqi

pairs = [
  JieqiPair(1912, Jieqi.小雪),
  JieqiPair(1913, Jieqi.秋分),
  JieqiPair(1917, Jieqi.大雪),
  JieqiPair(1927, Jieqi.白露),
  JieqiPair(1928, Jieqi.夏至),
  JieqiPair(1979, Jieqi.大寒),
]

results = [jieqi_moment(p.year, p.jq) for p in pairs]

moments = [r.moment for r in results]

moments_ut1_8 = [m + timedelta(hours=8) for m in moments]

moments_ut1_8

[datetime.datetime(1912, 11, 22, 23, 48, 14, 236353),
 datetime.datetime(1913, 9, 23, 23, 52, 48, 550398),
 datetime.datetime(1917, 12, 8, 0, 1, 5, 384493),
 datetime.datetime(1927, 9, 9, 0, 5, 30, 851009),
 datetime.datetime(1928, 6, 22, 0, 6, 27, 473321),
 datetime.datetime(1979, 1, 20, 23, 59, 56, 815078)]

* Encode the lunar year info for years 1600 - 2200
  * For years available using algo1, we use algo1
  * For other years, algo2 is used

In [11]:
algo1_range = common.get_supported_lunar_year_range(common.LunarAlgo.ALGO_1)
algo2_range = common.get_supported_lunar_year_range(common.LunarAlgo.ALGO_2)

print(f'Algo1 range: {algo1_range}')
print(f'Algo2 range: {algo2_range}')

def in_algo1_range(year: int) -> bool:
  return algo1_range.start <= year <= algo1_range.end

def in_algo2_range(year: int) -> bool:
  return algo2_range.start <= year <= algo2_range.end

Algo1 range: SupportedLunarYearRange(start=1901, end=2099)
Algo2 range: SupportedLunarYearRange(start=410, end=5000)


In [12]:
years = list(range(1600, 2200))

def find_algo(year) -> common.LunarAlgo:
  if in_algo1_range(year):
    return common.LunarAlgo.ALGO_1
  elif in_algo2_range(year):
    return common.LunarAlgo.ALGO_2
  else:
    raise ValueError(f'Year {year} is not in any of the supported ranges.')
  
algos = list(map(find_algo, years))

infos = list(map(common.get_lunar_year_info, algos, years))

In [13]:
from datetime import date

def encode_info(info: common.LunarYearInfo) -> int:
  # Find the offset from the first day in the Gregorian year.
  first_gregorian_day = (info.first_day - date(info.first_day.year, 1, 1)).days

  encoded = 0
  encoded |= (first_gregorian_day << 17)

  encoded |= (info.leap_month << 13)

  for i, month_len in enumerate(info.month_lengths):
    assert month_len in (29, 30)
    if month_len == 30:
      encoded |= (1 << i)

  return encoded

In [14]:
encoded = list(map(encode_info, infos))

In [15]:
query_table = dict(zip(years, encoded))

print(hex(query_table[1901]))
print(hex(query_table[1902]))
print(hex(query_table[1903]))
print(hex(query_table[2099]))

0x620752
0x4c0ea5
0x38b64a
0x28549b


In [16]:
# Generate C++ code.
# Print 10 entries in a row.

from itertools import batched

encoded_hex_str = [f'0x{e:x},' for e in encoded]

for batch in batched(encoded_hex_str, 10):
  print(' '.join(batch))

0x5a0ba4, 0x420b49, 0x2c7a93, 0x520a95, 0x3cf52d, 0x600556, 0x4a0ab5, 0x36d5aa, 0x5c05d2, 0x440da5,
0x309d4a, 0x560d4a, 0x400a96, 0x28552e, 0x4e0556, 0x38cab5, 0x5e0ad5, 0x4806d2, 0x328ea5, 0x580f25,
0x44064a, 0x2a6c97, 0x500a9b, 0x3d155a, 0x62056a, 0x4a0b69, 0x36b752, 0x5c0b52, 0x460b25, 0x2e964b,
0x540a4d, 0x3e04ab, 0x28455b, 0x4c05ad, 0x38eb69, 0x5e0da9, 0x4a0d92, 0x32bd25, 0x580d25, 0x420a55,
0x2c74ad, 0x5002b6, 0x3b65b5, 0x6006d5, 0x4c0ec9, 0x36be92, 0x5c0e92, 0x460d26, 0x308a56, 0x520a57,
0x3e04d6, 0x2826d5, 0x4e06d5, 0x38d6c9, 0x5e0749, 0x480693, 0x32b52b, 0x56052b, 0x400a5b, 0x2c755a,
0x52056a, 0x3b1b65, 0x620ba4, 0x4c0b49, 0x36da95, 0x5a0a95, 0x44052d, 0x2e8aad, 0x540ab5, 0x3e05aa,
0x284ba5, 0x4e0da5, 0x3afd4a, 0x5e0e4a, 0x480c96, 0x32b52e, 0x580556, 0x400ab5, 0x2c75b2, 0x5206d2,
0x3d0ea5, 0x600725, 0x4a064b, 0x34cc97, 0x5a0cab, 0x44055a, 0x2e8ad6, 0x540b69, 0x400752, 0x287725,
0x4e0b25, 0x38fa4b, 0x5e0a4d, 0x4604ab, 0x30a55b, 0x5605ad, 0x420baa, 0x2c7b52, 0x520d92, 0x3cfd25,
